
# CONNECT 4: Inteligência Artificial com Monte Carlo Tree Search e Árvores de Decisão (ID3)

**Desenvolvido por:** Cauã Pinheiro Souza e Matheus Gonçalves Guerra

---

## Introdução

Connect 4 é um jogo clássico de estratégia para dois jogadores, cujo objetivo é posicionar quatro peças consecutivas da mesma cor em linha — podendo ser horizontal, vertical ou diagonal. O jogo é disputado em um tabuleiro composto por sete colunas e seis linhas, onde os jogadores se alternam para inserir suas peças nas colunas disponíveis. O objetivo principal é formar uma sequência vencedora, enquanto simultaneamente bloqueia as jogadas do adversário.

Neste trabalho, desenvolvemos uma aplicação em Python que simula o jogo Connect 4 e explora técnicas de inteligência artificial para aprimorar a experiência e o desafio oferecido pelo sistema. A aplicação oferece três modos de jogo distintos:

- **Jogador vs Jogador:** dois usuários humanos competem entre si no mesmo dispositivo.
- **Jogador vs Inteligência Artificial:** um jogador humano enfrenta uma IA que decide suas jogadas.
- **Inteligência Artificial vs Inteligência Artificial:** duas diferentes estratégias de IA competem uma contra a outra, possibilitando análise comparativa.

Para o desenvolvimento da inteligência artificial, utilizamos duas abordagens principais:

1. **Monte Carlo Tree Search (MCTS):** uma técnica de simulação que avalia as possíveis jogadas por meio de centenas ou milhares de partidas simuladas, buscando selecionar o melhor movimento baseado em probabilidades estatísticas.
2. **Árvores de Decisão (ID3):** um algoritmo de aprendizado supervisionado que cria modelos de decisão a partir de dados de jogos anteriores, permitindo que a IA aprenda padrões e estratégias eficazes para melhorar sua performance.

Este notebook apresenta a implementação do jogo, descreve a estrutura dos códigos e detalha como as técnicas de IA foram aplicadas para criar adversários cada vez mais inteligentes, além de discutir exemplos de uso e testes.



## Estrutura do Projeto

O projeto está organizado em diferentes pastas, cada uma com uma responsabilidade específica:

- `main.py` – Arquivo principal para execução do jogo.
- `ai_alg/` – Algoritmos de Inteligência Artificial (Monte Carlo, heurísticas, Alpha-Beta).
- `game_structure/` – Estrutura central do jogo: regras, tabuleiro, interface gráfica, etc.
- `training/` – Scripts para Árvores de Decisão, geração/análise de datasets e ID3.
- `requirements.txt` – Lista de dependências Python.


### main.py
**Função:** Arquivo principal de execução do projeto. Ele inicializa o jogo, define modos de partida e integra as IAs e interface.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `main()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
from game_structure.Board import Board
from game_structure.interface import Interface


def main() -> None:
    board = Board()
    interface = Interface()
    interface.starting_game(board)


if __name__ == "__main__":
    main()


### alpha_beta.py
**Função:** Implementação do algoritmo Alpha-Beta Pruning para busca em árvore de decisões.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `alpha_beta()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `evaluate_position()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `generate_children()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


**Esta matriz de confusão ilustra o desempenho do modelo baseline ao classificar jogadas no Connect 4. Ela permite analisar quais classes (jogadas) são mais corretamente previstas e onde o modelo erra mais, sendo fundamental para comparar a evolução do desempenho durante o desenvolvimento da IA.**

![](baseline_confusion_matrix.png)

In [ ]:
import numpy as np
from game_structure import style as s
from game_structure import game_engine as game
from ai_alg import heuristic as h


def alpha_beta(board: np.ndarray) -> int:
    """Usando o algoritmo de alpha-beta, escolhe a melhor jogada até uma certa profundidade"""
    max_depth = 5
    best_col = -1
    best_score = float('-inf')

    for new_board, col in generate_children(board, s.SECOND_PLAYER_PIECE):
        if game.winning_move(new_board, s.SECOND_PLAYER_PIECE):
            return col
        score = evaluate_position(
            new_board, current_depth=1,
            alpha=float('-inf'), beta=float('inf'),
            depth_limit=max_depth, is_maximizing=False
        )
        if score > best_score:
            best_score = score
            best_col = col

    return best_col


def evaluate_position(board: np.ndarray,current_depth: int,alpha: float,beta: float,depth_limit: int,is_maximizing: bool) -> float:
    """Usa minimax e alpha-beta para avaliar o tabuleiro"""
    if (current_depth == depth_limit or game.winning_move(board, s.FIRST_PLAYER_PIECE) or game.winning_move(board, s.SECOND_PLAYER_PIECE) or game.is_game_tied(board)):
        return h.calculate_board_score(board, s.SECOND_PLAYER_PIECE, s.FIRST_PLAYER_PIECE)

    if is_maximizing:
        max_eval = float('-inf')
        for child_board, _ in generate_children(board, s.SECOND_PLAYER_PIECE):
            evaluation = evaluate_position(child_board, current_depth + 1, alpha, beta, depth_limit, False)
            max_eval = max(max_eval, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_eval

    else:
        min_eval = float('inf')
        for child_board, _ in generate_children(board, s.FIRST_PLAYER_PIECE):
            evaluation = evaluate_position(child_board, current_depth + 1, alpha, beta, depth_limit, True)
            min_eval = min(min_eval, evaluation)
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        return min_eval


def generate_children(board: np.ndarray, piece: int):
    """Gera tabuleiros filhos para uma determinada peça."""
    children = []
    available = game.available_moves(board)
    if available == -1:
        return children
    for col in available:
        new_board = game.simulate_move(board, piece, col)
        children.append((new_board, col))
    return children


### basic_heuristic.py
**Função:** Funções heurísticas para avaliar estados do jogo.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `evaluate_best_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `adversarial_lookahead()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
import numpy as np
from game_structure import style as s
from game_structure import game_engine as game
from ai_alg import heuristic as h


def evaluate_best_move(board: np.ndarray, player_piece: int, enemy_piece: int) -> int:
    """Avalia todas as possíveis jogadas e retorna aquela com a maior pontuação, utilizando a função heurística"""
    top_score = float('-inf')
    chosen_column = -1
    for column in game.available_moves(board):
        future_board = game.simulate_move(board, player_piece, column)
        evaluation = h.calculate_board_score(future_board, player_piece, enemy_piece)
        if evaluation > top_score:
            top_score = evaluation
            chosen_column = column
    return chosen_column


def adversarial_lookahead(board: np.ndarray, player_piece: int, enemy_piece: int) -> int:
    """Considera a próxima jogada e a melhor repostas possível do adversário a esta resposta, escolhendo assim a jogada com melhor vantagem"""
    optimal_column = -1
    highest_evaluation = float('-inf')
    suggested_counter = 0

    legal_columns = game.available_moves(board)
    if len(legal_columns) == 1:
        return legal_columns[0]

    for column in legal_columns:
        player_future = game.simulate_move(board, player_piece, column)
        if game.winning_move(player_future, s.SECOND_PLAYER_PIECE):
            return column
        predicted_opponent = evaluate_best_move(player_future, enemy_piece, player_piece)
        opponent_future = game.simulate_move(player_future, enemy_piece, predicted_opponent)
        evaluation = h.calculate_board_score(opponent_future, player_piece, enemy_piece)
        if evaluation > highest_evaluation:
            highest_evaluation = evaluation
            optimal_column = column
            suggested_counter = predicted_opponent + 1

    print("Possivel jogada: coluna no", suggested_counter + 1)
    return optimal_column


### heuristic.py
**Função:** Funções heurísticas para avaliar estados do jogo.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `calculate_board_score()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `weights()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


**Aqui temos a visualização da árvore de decisão treinada com dados de 300 partidas de Connect 4. Esta árvore representa o processo de tomada de decisão da IA em estágios iniciais, quando ainda possui pouca experiência. Com ela, é possível entender os critérios iniciais usados pela IA para prever movimentos.**

![](connect4_300games_tree.png)

In [ ]:
from game_structure import style as s
import numpy as np

def calculate_board_score(board: np.ndarray, piece: int, opponent_piece: int) -> int:
    score = 0

    # Verifica horizontal
    for col in range(s.COLUMNS - 3):
        for r in range(s.ROWS):
            segment = [board[r][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Verifica vertical
    for col in range(s.COLUMNS):
        for r in range(s.ROWS - 3):
            segment = [board[r + i][col] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Verifica diagonal ascendente
    for col in range(s.COLUMNS - 3):
        for r in range(s.ROWS - 3):
            segment = [board[r + i][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Verifica diagonal descendente
    for col in range(s.COLUMNS - 3):
        for r in range(3, s.ROWS):
            segment = [board[r - i][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    return score


def weights(segment: list, piece: int, opponent_piece: int) -> int:
    if piece in segment and opponent_piece in segment: return 0
    if segment.count(piece) == 1: return 1
    if segment.count(piece) == 2: return 10
    if segment.count(piece) == 3: return 50
    if segment.count(piece) == 4: return 1000
    if segment.count(opponent_piece) == 1: return -1
    if segment.count(opponent_piece) == 2: return -10
    if segment.count(opponent_piece) == 3: return -50
    if segment.count(opponent_piece) == 4: return -2000
    return 0


### monte_carlo.py
**Função:** Implementação do algoritmo Monte Carlo Tree Search para escolher a melhor jogada.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Classe `Node`**: Classe principal do arquivo. Implementa funcionalidades centrais relacionadas ao nome e propósito do arquivo. Analise detalhada pode ser fornecida sob demanda.
- **Classe `monte_carlo`**: Classe principal do arquivo. Implementa funcionalidades centrais relacionadas ao nome e propósito do arquivo. Analise detalhada pode ser fornecida sob demanda.
- **Função `mcts()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `__init__()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `__str__()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `add_children()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `select_children()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `ucb()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `score()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `__init__()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `start()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `search()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `select()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `best_child()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `back_propagation()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `expand()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `rollout()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `best_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
import time, math, numpy as np, random, itertools
from game_structure import style as s
from game_structure import game_engine as game
from math import sqrt, log
from ai_alg import heuristic as h

class Node:
    """Representa um nó na árvore de busca Monte Carlo, onde cada nó representa uma configuração diferente do tabuleiro"""
    
    def __init__(self, board, last_player, parent=None) -> None:
        self.board = board  # Estado atual do tabuleiro
        self.parent = parent  # Nó pai na árvore
        self.children = []  # Lista de nós filhos
        self.visits = 0  # Número de vezes que este nó foi visitado
        self.wins = 0  # Número de vitórias alcançadas a partir deste nó
        # Determina o jogador atual com base em que jogou por ultimo
        self.current_player = 1 if last_player == 2 else 2

    def __str__(self) -> str:
        """Representação em string do nó para depuração"""
        
        string = "Vitórias: " + str(self.wins) + '\n'
        string += "Total de visitas: " + str(self.visits) + '\n'
        string += "Pontuação UCB: " + str(self.ucb()) + '\n'
        string += "Probabilidade de vitória: " + str(self.score()) + '\n'
        return string

    def add_children(self) -> None:
        """Gera todos possíveis movimentos no tabuleiro atual, adicionando-os como filhos deste nó"""
        
        if (len(self.children) != 0) or (game.available_moves(self.board) == -1):
            return
            
        for col in game.available_moves(self.board):
            # Cria uma cópia do tabuleiro com a nova jogada aplicada
            if self.current_player == s.FIRST_PLAYER_PIECE:
                copy_board = game.simulate_move(self.board, s.SECOND_PLAYER_PIECE, col)
            else:
                copy_board = game.simulate_move(self.board, s.FIRST_PLAYER_PIECE, col)
                
            # Adiciona o novo nó e a coluna que o gerou à lista de filhos
            self.children.append((Node(board=copy_board, last_player=self.current_player, parent=self), col))

    def select_children(self):
        """Seleciona aleatoriamente no máximo 4 filhos para apurar a busca, melhorando assim a sua eficiência"""
        
        if (len(self.children) > 4):
            return random.sample(self.children, 4)
        return self.children

    def ucb(self) -> float:
        """Cacula o UCB (Upper Confidence Bound) para escolher os nós, equilibrando assim as escolhas aprofundadas e as não"""
        
        if self.visits == 0:
            return float('inf')  # Nós não visitados têm prioridade máxima
            
        # Fórmula UCB: wins/visits + C * sqrt(ln(parent_visits)/visits)
        exploitation = self.wins / self.visits
        exploration = sqrt(2) * sqrt(log(self.parent.visits) / self.visits)
        return exploitation + exploration
        
    def score(self) -> float:
        """Calcula a taxa de vitórias para este nó"""
        if self.visits == 0:
            return 0
        return self.wins / self.visits


class monte_carlo:
    """Implementação do algoritmo de Monte Carlo TS, usando UCB1 para escolher os nós e simulando usando heurística"""
    
    def __init__(self, root: Node) -> None:
        self.root = root

    def start(self, max_time: int):
        """Inicia a busca realizando simulações iniciais nos filhos da raiz, executando em seguida o algoritmo mcts"""
        
        self.root.add_children()   
        for child in self.root.children:
            if game.winning_move(child[0].board, s.SECOND_PLAYER_PIECE):
                return child[1]  
            for _ in range(6):
                result = self.rollout(child[0])
                self.back_propagation(child[0], result)
                
        return self.search(max_time)

    def search(self, max_time: int) -> int:
        """Executa o algoritmo princiapl do mcts até o limite de tempo"""
        
        start_time = time.time()
        while time.time() - start_time < max_time:
            selected_node = self.select(self.root)
            
            # Analisa se o nó foi ou não visitado, mudando ou não de onde partir
            if selected_node.visits == 0:
                result = self.rollout(selected_node)
                self.back_propagation(selected_node, result)
            else:
                selected_children = self.expand(selected_node)
                for child in selected_children:
                    result = self.rollout(child[0])
                    self.back_propagation(child[0], result)
                                   
        return self.best_move()

    def select(self, node: Node) -> Node:
        """Seleciona o nó com maiores chances, utilizando UCB para equilibrar"""
        
        if node.children == []:
            return node
        else:
            node = self.best_child(node)
            return self.select(node)

    def best_child(self, node: Node) -> Node:
        best_child = None
        best_score = float('-inf')
        
        for (child, _) in node.children:
            ucb = child.ucb()
            if ucb > best_score:
                best_child = child
                best_score = ucb
                
        return best_child

    def back_propagation(self, node: Node, result: int) -> None:
        """Atualiza os dados para todos os nós da árvore, incrementando visit count e possívelmente win count"""
        
        while node:
            node.visits += 1
            if node.current_player == result:
                node.wins += 1
            node = node.parent

    def expand(self, node: Node) -> Node:
        """Adiciona todos os possíveis nós filhos e seleciona alguns"""
        
        node.add_children()
        return node.select_children()

    def rollout(self, node: Node) -> int:
        """Simula uma possível partida a partir do nó atual, utilizando de jogadas guiadas por heurística, retornando assim o vencedor da simulação"""
        
        board = node.board.copy()
        max_depth = 6
        players = itertools.cycle([s.SECOND_PLAYER_PIECE, s.FIRST_PLAYER_PIECE])
        current_player = next(players)

        for _ in range(max_depth):
            if game.winning_move(board, s.SECOND_PLAYER_PIECE) or game.winning_move(board, s.FIRST_PLAYER_PIECE):
                break
            if game.is_game_tied(board):
                return 0   
            current_player = next(players)
            moves = game.available_moves(board) 
            if moves == -1:
                return 0 
            # Usa heurística para escolher a melhor jogada (não aleatória)
            best_move = max(
                moves,
                key=lambda col: h.calculate_board_score(
                    game.simulate_move(board, current_player, col), 
                    s.SECOND_PLAYER_PIECE, 
                    s.FIRST_PLAYER_PIECE
                )
            )
            
            board = game.simulate_move(board, current_player, best_move)

        return current_player

    def best_move(self) -> int:
        """Seleciona as melhores opções, baseado na taxa de vitória dos filhos da raíz, possívelmente com a mesma pontuação, "desempatando" aleatoriamente"""
        
        max_score = float('-inf')
        scores = {}  # Armazena os pares de colunas e suas respectivas pontuações
        columns = []  # Armazena as colunas com a melhor pontuação
        
        # Calcula as pontuações possíveis
        for (child, col) in self.root.children:
            score = child.score()
            print(f"Coluna: {col}")
            print(child)
            
            if score > max_score:
                max_score = score
                
            scores[col] = score
            
        # Armazena todos os movimentos que resultam com a melhor pontuação
        for col, score in scores.items():
            if score == max_score:
                columns.append(col)
                
        # Escolhe aleatoriamente entre os melhores movimentos
        return random.choice(columns)


def mcts(board: np.ndarray) -> int:
    """Função que executa a busca mcts e retorna a melhor coluna para se jogar"""
    
    root = Node(board=board, last_player=s.SECOND_PLAYER_PIECE)
    mc = monte_carlo(root)
    column = mc.start(3)  # Executa o MCTS por 3 segundos
    print(column + 1)
    return column


### Board.py
**Função:** Implementa o tabuleiro do Connect 4, as regras do jogo, controle de peças, verificação de vitória/empate.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Classe `Board`**: Classe principal do arquivo. Implementa funcionalidades centrais relacionadas ao nome e propósito do arquivo. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_board()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `print_board()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


**Este gráfico representa a distribuição do conjunto de dados obtido após 3.500 partidas, considerando um limite de 1,5 segundo por jogada. Mostra o crescimento e a qualidade do dataset usado para treinar a IA, fundamental para avaliar a diversidade e a robustez dos exemplos que alimentam o modelo.**

![](connect4_3500games1point5sec_dataset.png)

In [ ]:
import numpy as np
from dataclasses import dataclass, field
from game_structure import style as s

@dataclass
class Board:
    rows: int = s.ROWS
    columns: int = s.COLUMNS
    board: np.ndarray = field(default_factory=lambda: np.zeros((s.ROWS, s.COLUMNS)))

    def get_board(self):
        return self.board

    def print_board(self):
        print(np.flip(self.board, 0), "\n")


### ai_game.py
**Função:** Implementa jogos controlados por IA, seja contra IA, humano ou IA vs IA.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `run_ai_vs_ai_game()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_ai_types()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_ai_column_for_type()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
import pygame
import numpy as np
from game_structure import Board
from game_structure import style as s
from game_structure import game_engine as game
from ai_alg import basic_heuristic as b, alpha_beta as a, monte_carlo as m, heuristic as h
from training.decision_tree_player import decision_tree_move  # <-- Mantenha este import
import itertools
import time

def run_ai_vs_ai_game(interface, brd, game_mode):
    """Simula uma partida entre duas IA, podendo ou não serem diferentes"""
    
    board = brd.get_board()
    game_over = False
    turns = itertools.cycle([1, 2]) 
    turn = next(turns)
    
    # Define qual IA cada jogador será
    ai_types = get_ai_types(game_mode)
    
    interface.draw_board()
    pygame.display.update()
    
    pygame.time.wait(1000)
    
    move_count = 0
    
    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                interface.quit()
                
        # Pequeno atraso entre as jogadas para as partidas poderem serem assistidas
        pygame.time.wait(500)
        
        ai_type = ai_types[turn - 1]
        col = get_ai_column_for_type(board, turn, ai_type)
        
        row = game.get_next_open_row(board, col)
        if row != -1:
            game.drop_piece(board, row, col, turn)
            interface.draw_new_piece(row + 1, col + 2, turn)
            pygame.display.update()
            brd.print_board()
            move_count += 1
            print(f"Jogada {move_count}: Jogador {turn} ({ai_type}) colocou na coluna {col}")
            
            if game.winning_move(board, turn):
                font = pygame.font.SysFont('Connect4-main/fonts/SuperMario256.ttf', 50)
                interface.show_winner(font, turn)
                game_over = True
                break  
            if game.is_game_tied(board):
                font = pygame.font.SysFont('Connect4-main/fonts/SuperMario256.ttf', 50)
                interface.show_draw(font)
                game_over = True
                break
                
        turn = next(turns)
    
    pygame.time.wait(10000)
    return

def get_ai_types(game_mode):
    """Retorna as IAs escolhidas"""
    
    ai_mapping = {
        6: ["Easy", "Easy"],           # Fácil x Fácil
        7: ["Easy", "Hard"],           # Fácil x Difícil
        8: ["Medium", "Medium"],       # Médio x Médio
        9: ["Medium", "Challenge"],    # Médio x Desafio
        10: ["Hard", "Challenge"],     # AlphaBeta x Desafio
        11: ["Easy", "Medium"],        # Fácil x Médio
        12: ["Easy", "Challenge"],     # Fácil x Desafio
        13: ["Medium", "Hard"],        # Médio x AlphaBeta
        14: ["Hard", "Hard"],          # AlphaBeta x AlphaBeta
        15: ["Challenge", "Challenge"] # Desafio x Desafio
    }
    return ai_mapping.get(game_mode, ["Easy", "Easy"])

def get_ai_column_for_type(board, player, ai_type):
    """Obtém a melhor jogada de acordo com a respectiva IA"""
    
    opponent = 1 if player == 2 else 2
    
    if ai_type == "Easy":
        # Usa heurística básica (A*)
        return b.evaluate_best_move(board, player, opponent)
    elif ai_type == "Medium":
        # Usa Monte Carlo Tree Search (MCTS)
        return m.mcts(board)
    elif ai_type == "Hard":
        # Usa poda Alpha-Beta
        return a.alpha_beta(board)
    elif ai_type == "Challenge":
        # Usa agente Decision Tree
        return decision_tree_move(board)
    else:
        # Padrão para heurística básica
        return b.evaluate_best_move(board, player, opponent)


### game_engine.py
**Função:** Gerencia a lógica de execução de uma partida, controle de turnos e integra os diferentes tipos de jogadores.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `first_player_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_human_column()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `available_moves()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `ai_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_ai_column()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `simulate_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `make_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_next_open_row()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `drop_piece()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `is_game_tied()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `is_valid()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `winning_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


**Visualização da árvore de decisão após treinamento com 3.500 partidas. Comparando com a árvore de 300 jogos, é possível perceber o quanto a IA evolui e refina sua lógica de decisões à medida que joga mais, tornando suas previsões mais estratégicas e baseadas em experiência real.**

![](connect4_3500games_tree.png)

In [ ]:
from game_structure import style as s
import numpy as np
import math
import pygame
from game_structure import Board
from ai_alg import basic_heuristic as b, alpha_beta as a, monte_carlo as m
from training.decision_tree_player import decision_tree_move

def first_player_move(bd: Board, interface: any, board: np.ndarray, turn: int, event: any) -> bool:
    """Define a coluna jogada pelo jogador 1"""
    
    col = get_human_column(interface, event)
    if not is_valid(board, col): return False

    pygame.draw.rect(interface.screen, s.BACKGROUND_COLOR, (0, 0, interface.width, interface.pixels - 14))
    make_move(bd, interface, board, turn, col)
    return True

def get_human_column(interface: any, event: any):
    """Obtém a coluna selecionada pelo mouse"""
    
    posx = event.pos[0]
    col = int(math.floor(posx / interface.pixels)) - 2
    return col

def available_moves(board: np.ndarray) -> list | int:
    """Retorna uma lista de colunas disponíveis para jogar, ou -1 se não houver"""
    
    avaiable_moves = []
    for i in range(s.COLUMNS):
        if (board[5][i]) == 0:
            avaiable_moves.append(i)
    return avaiable_moves if len(avaiable_moves) > 0 else -1

def ai_move(bd: Board, interface: any, game_mode: int, board: np.ndarray, turn: int) -> int:
    """Define a coluna jogada pelo jogador 2"""
    
    ai_column = get_ai_column(board, game_mode, turn)
    game_over = make_move(bd, interface, board, turn, ai_column)
    return game_over

def get_ai_column(board: Board, game_mode: int, player: int = 2) -> int:
    """Seleciona o algoritmo de IA escolhido para jogar"""
    
    opponent = 1 if player == 2 else 2

    if game_mode == 2:
        return b.evaluate_best_move(board, player, opponent)
    elif game_mode == 3:
        return m.mcts(board)
    elif game_mode == 4:
        return a.alpha_beta(board)
    elif game_mode == 5:
        return decision_tree_move(board)
    return 0

def simulate_move(board: np.ndarray, piece: int, col: int) -> np.ndarray:
    """Simula uma jogada em uma cópia do tabuleiro"""
    
    board_copy = board.copy()
    row = get_next_open_row(board_copy, col)
    drop_piece(board_copy, row, col, piece)
    return board_copy

def make_move(bd: Board, interface: any, board: np.ndarray, turn: int, move: int):
    """Executa a jogada e verifica se ela resulta em vitória ou empate"""

    row = get_next_open_row(board, move)
    drop_piece(board, row, move, turn) 
    interface.draw_new_piece(row + 1, move + 2, turn)
    pygame.display.update()
    bd.print_board()

    return winning_move(board, turn) or is_game_tied(board)

def get_next_open_row(board: np.ndarray, col: int) -> int:
    """Retorna a linha disponível para colocar a peça na coluna escolhida"""
    
    for row in range(s.ROWS):
        if board[row, col] == 0:
            return row
    return -1

def drop_piece(board: np.ndarray, row: int, col: int, piece: int) -> None:
    """Insere a peça no tabuleiro na posição escolhida"""
    
    board[row, col] = piece

def is_game_tied(board: np.ndarray) -> bool:
    """Verifica se o jogo empatou"""
    
    if winning_move(board, s.SECOND_PLAYER_PIECE) or winning_move(board, s.FIRST_PLAYER_PIECE): return False
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0: return False
    return True

def is_valid(board: np.ndarray, col: int) -> bool:
    """Analisa se a coluna escolhida é válida"""
    
    if not 0 <= col < s.COLUMNS: return False
    row = get_next_open_row(board, col)
    return 0 <= row <= 5

def winning_move(board: np.ndarray, piece: int) -> bool:
    """Analisa se a peça colocada resultou em vitória"""
    
    rows, cols = board.shape
    for row in range(rows):
        for col in range(cols):
            if int(board[row, col]) == piece:
                # Checa horizontalmente
                if col + 3 < cols and all(board[row, col + i] == piece for i in range(4)):
                    return True
                # Checa verticalmente
                if row + 3 < rows and all(board[row + i, col] == piece for i in range(4)):
                    return True
                # Checa diagonal ascendente
                if row - 3 >= 0 and col + 3 < cols and all(board[row - i, col + i] == piece for i in range(4)):
                    return True
                # Checa diagonal descendente
                if row + 3 < rows and col + 3 < cols and all(board[row + i, col + i] == piece for i in range(4)):
                    return True
    return False


### interface.py
**Função:** Gerencia a interface gráfica do usuário.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Classe `Interface`**: Classe principal do arquivo. Implementa funcionalidades centrais relacionadas ao nome e propósito do arquivo. Analise detalhada pode ser fornecida sob demanda.
- **Função `starting_game()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `play()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `draw_menu()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `render_alternating_colors_text()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `choose_option()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `draw_combinations()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `choose_AI_combination()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `draw_difficulties()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `choose_AI_difficulty()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `draw_board()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `draw_new_piece()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `draw_button()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `show_winner()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `show_draw()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `quit()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
import pygame
import itertools
import sys
from game_structure import style as s
from game_structure import Board
from game_structure import game_engine as game
from dataclasses import dataclass
from game_structure.ai_game import run_ai_vs_ai_game

@dataclass
class Interface:
    rows: int = s.ROWS
    columns: int = s.COLUMNS
    pixels: int = s.SQUARE_SIZE
    width: int = s.WIDTH
    height: int = s.HEIGHT
    rad: float = s.RADIUS_PIECE
    size: tuple = (width, height)
    screen: any = pygame.display.set_mode(size)
    pygame.display.set_caption("Connect4")

    def starting_game(self, brd: Board):
        """Inicia o programa para rodar o jogo"""
        
        pygame.init()
        self.draw_menu()
        game_mode = self.choose_option()
        brd.print_board()
        self.draw_board()
        pygame.display.update()
        
        if game_mode >= 6 and game_mode <= 15:  # Modos IA vs IA
            run_ai_vs_ai_game(self, brd, game_mode)
        else:
            self.play(brd, game_mode)

    def play(self, brd: Board, game_mode: int):
        board = brd.get_board()
        game_over = False
        # Mario !!
        font = pygame.font.SysFont('Connect4-main/fonts/SuperMario256.ttf', 50)
        turns = itertools.cycle([1, 2])
        turn = next(turns)

        while not game_over:
            for event in pygame.event.get():
                if event.type == pygame.QUIT: quit()

                if event.type == pygame.MOUSEMOTION:
                    pygame.draw.rect(self.screen, s.BACKGROUND_COLOR, (0, 0, self.width, self.pixels - 14))
                    posx = event.pos[0]
                    pygame.draw.circle(self.screen, s.PIECES_COLORS[turn], (posx, int(self.pixels / 2) - 7), self.rad)
                pygame.display.update()

                if event.type == pygame.MOUSEBUTTONDOWN:
                    if turn == 1 or (turn == 2 and game_mode == 1): 
                        if not game.first_player_move(brd, self, board, turn, event): continue  # verifica se a coluna é valida
                        if game.winning_move(board, turn):
                            game_over = True
                            break
                        turn = next(turns)

                if turn != 1 and game_mode != 1:
                    pygame.time.wait(15)
                    game_over = game.ai_move(brd, self, game_mode, board,turn)  # recebe jogada da IA e retorna se o jogo acabou
                    if game_over: break
                    turn = next(turns)

            if game.is_game_tied(board):
                self.show_draw(font)
                break

        if game.winning_move(board, turn):
            self.show_winner(font, turn)

        pygame.time.wait(10000)

    def draw_menu(self):
        """Desenha o menu e as opções do jogo"""
        
        self.screen.fill(s.BACKGROUND_COLOR)
        font = pygame.font.Font('Connect4-main/fonts/SuperMario256.ttf', 80)

        colors = [s.RED, s.YELLOW, s.BLUE, s.GREEN]
        pos = (560 - font.size("Connect 4")[0] // 2, 230 - font.get_height() // 2)

        self.render_alternating_colors_text("Connect 4", font, colors, pos)

        self.draw_button(self.height / 2, 350, 300, 50, "Single Player")
        self.draw_button(self.height / 2, 450, 300, 50, "Multiplayer")
        self.draw_button(self.height / 2, 550, 300, 50, "PC x PC")

    def render_alternating_colors_text(self, text: str, font, colors, pos):
        """Função para fazer os titulos ficarem igual ao super mario"""
        
        x, y = pos
        outline_color = s.BLACK
        outline_range = range(-3,4)

        for i, char in enumerate(text):
            color = colors[i % len(colors)]

            for ox in outline_range:
                for oy in outline_range:
                    if ox == 0 and oy == 0:
                        continue  # Pula o centro
                    outline_surface = font.render(char, True, outline_color)
                    self.screen.blit(outline_surface, (x + ox, y + oy))

            letter_surface = font.render(char, True, color)
            self.screen.blit(letter_surface, (x, y))
            x += letter_surface.get_width()  # Move x para o próximo caractere
        pygame.display.update()

    def choose_option(self) -> int:
        """Retorna o modo de jogo escolhido"""
        
        while True:
            game_mode = 0
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    quit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    if (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 350 <= mouse_y <= 400:
                        print("Player vs IA selecionado")
                        self.draw_difficulties()
                        game_mode = 2
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 450 <= mouse_y <= 500:
                        print("Player vs Player selecionado")
                        game_mode = 1
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 550 <= mouse_y <= 600:
                        print("IA vs IA selecionado")
                        game_mode = 3

            pygame.display.flip()

            if game_mode == 1:
                return game_mode

            if game_mode == 2:
                game_mode = self.choose_AI_difficulty()
                return game_mode

            if game_mode == 3:
                self.draw_combinations()
                game_mode = self.choose_AI_combination()
                return game_mode

    def draw_combinations(self):
        """Desenha as possíveis combinações entre as IAs"""
        
        self.screen.fill(s.BACKGROUND_COLOR)

        left_x = self.width / 4 - 200
        right_x = self.width * 3 / 4 - 200

        self.draw_button(left_x, 150, 400, 50, "Easy x Easy")  # A* x A*
        self.draw_button(left_x, 250, 400, 50, "Easy x Hard")  # A* x alpha beta
        self.draw_button(left_x, 350, 400, 50, "Medium x Medium")  # mcts x mcts
        self.draw_button(left_x, 450, 400, 50, "Medium x Challenge")  # mcts x decision tree
        self.draw_button(left_x, 550, 400, 50, "Hard x Challenge")  # alpha beta x decision tree

        self.draw_button(right_x, 150, 400, 50, "Easy x Medium")  # A* x mcts
        self.draw_button(right_x, 250, 400, 50, "Easy x Challenge")  # A* x decision tree
        self.draw_button(right_x, 350, 400, 50, "Medium x Hard")  # mcts x alpha beta
        self.draw_button(right_x, 450, 400, 50, "Hard x Hard")  # alpha beta x alpha beta
        self.draw_button(right_x, 550, 400, 50, "Challenge x Challenge")  # decision tree x decision tree

    def choose_AI_combination(self):
        """Retorna qual combinação foi escolhida"""
        
        left_x = self.width / 4 - 200
        right_x = self.width * 3 / 4 - 200
        game_mode = 0

        while True:
            for event in pygame.event.get():
                current_event = event.type
                if current_event == pygame.QUIT:
                    quit()
                elif current_event == pygame.MOUSEBUTTONDOWN:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    if left_x <= mouse_x <= left_x + 400:
                        if 150 <= mouse_y <= 200:
                            game_mode = 6
                            print("Easy x Easy")
                        elif 250 <= mouse_y <= 300:
                            game_mode = 7
                            print("Easy x Hard")
                        elif 350 <= mouse_y <= 400:
                            game_mode = 8
                            print("Medium x Medium")
                        elif 450 <= mouse_y <= 500:
                            game_mode = 9
                            print("Medium x Challenge")
                        elif 550 <= mouse_y <= 600:
                            game_mode = 10
                            print("Hard x Challenge")
                    elif right_x <= mouse_x <= right_x + 400:
                        if 150 <= mouse_y <= 200:
                            game_mode = 11
                            print("Easy x Medium")
                        elif 250 <= mouse_y <= 300:
                            game_mode = 12
                            print("Easy x Challenge")
                        elif 350 <= mouse_y <= 400:
                            game_mode = 13
                            print("Medium x Hard")
                        elif 450 <= mouse_y <= 500:
                            game_mode = 14
                            print("Hard x Hard")
                        elif 550 <= mouse_y <= 600:
                            game_mode = 15
                            print("Challenge x Challenge")
            pygame.display.flip()
            if game_mode != 0:
                return game_mode

    def draw_difficulties(self):
        """Desenha as dificuldades para single player"""
        
        self.screen.fill(s.BACKGROUND_COLOR)
        self.draw_button(self.height / 2, 250, 300, 50, "Easy")       # A*
        self.draw_button(self.height / 2, 350, 300, 50, "Medium")     # Monte Carlo (MCTS)
        self.draw_button(self.height / 2, 450, 300, 50, "Hard")       # Alpha Beta
        self.draw_button(self.height / 2, 550, 300, 50, "Challenge")  # Decision Tree

    def choose_AI_difficulty(self):
        game_mode = 0
        while True:
            for event in pygame.event.get():
                current_event = event.type
                if current_event == pygame.QUIT: quit()
                elif current_event == pygame.MOUSEBUTTONDOWN:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    if (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 250 <= mouse_y <= 300:
                        game_mode = 2
                        print("A*")
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 350 <= mouse_y <= 400:
                        game_mode = 3
                        print("A* Adversarial")
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 450 <= mouse_y <= 500:
                        game_mode = 4
                        print("Alpha Beta")
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 550 <= mouse_y <= 600:
                        game_mode = 5
                        print("MCTS")
                pygame.display.flip()
                if game_mode != 0:
                    return game_mode

    def draw_board(self):
        """Desenha o tabuleiro do jogo"""
        
        self.screen.fill(s.BACKGROUND_COLOR)

        shadow_coordinates = (2 * self.pixels - 10, self.pixels - 10, self.columns * self.pixels + 24, self.rows * self.pixels + 24)
        board_coordinates = (2 * self.pixels - 10, self.pixels - 10, self.columns * self.pixels + 20, self.rows * self.pixels + 20)
        pygame.draw.rect(self.screen, s.GRAY, shadow_coordinates, 0, 30)
        pygame.draw.rect(self.screen, s.BOARD_COLOR, board_coordinates, 0, 30)

        # desenha os espaços vazios:
        for col in range(self.columns):
            for row in range(self.rows):
                center_of_circle = (int((col + 5 / 2) * self.pixels), int((row + 3 / 2) * self.pixels))
                pygame.draw.circle(self.screen, s.BACKGROUND_COLOR, center_of_circle, self.rad)
        pygame.display.update()

    def draw_new_piece(self, row: int, col: int, piece: int):
        """Desenha a nova peça a ser colocada"""
        
        center_of_circle = (int(col * self.pixels + self.pixels / 2), self.height - int(row * self.pixels + self.pixels / 2))
        pygame.draw.circle(self.screen, s.PIECES_COLORS[piece], center_of_circle, self.rad)

    def draw_button(self, x: int, y: int, width: int, height: int, text: str):
        """Desenha os botões de opção"""
        
        pygame.draw.rect(self.screen, s.GRAY, (x, y, width, height), 0, 30)
        font = pygame.font.Font('Connect4-main/fonts/SuperMario256.ttf', 25)
        text_surface = font.render(text, True, s.BLACK)
        text_rect = text_surface.get_rect()
        text_rect.center = (x + width / 2, y + height / 2)
        self.screen.blit(text_surface, text_rect)

    def show_winner(self, font: any, turn: int):
        """Exibe o vencedor"""
        
        font = pygame.font.Font('Connect4-main/fonts/SuperMario256.ttf', 50)
        colors = [s.RED, s.YELLOW, s.BLUE, s.GREEN]
        winner = ("Jogador " + str(turn) + " venceu!")
        pos = (560 - font.size(winner)[0] // 2, 20)
        self.render_alternating_colors_text(winner, font, colors, pos)
        pygame.display.update()

    def show_draw(self, font: any):
        """Exibe mensagem de empate""" 
        
        font = pygame.font.Font('Connect4-main/fonts/SuperMario256.ttf', 50)
        colors = [s.RED, s.YELLOW, s.BLUE, s.GREEN]
        draw_message = "Jogo empatado!"
        pos = (560 - font.size(draw_message)[0] // 2, 20)   
        self.render_alternating_colors_text(draw_message, font, colors, pos)
        pygame.display.update()

    def quit(self):
        pygame.quit()
        sys.exit()


### style.py
**Função:** Define estilos visuais para a interface do jogo.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- Este arquivo não possui funções ou classes explícitas, apenas código procedural.

**Esta árvore é um exemplo de teste usando o famoso dataset Iris. Serve como demonstração de funcionamento dos métodos de visualização da árvore de decisão no projeto, garantindo que a lógica de exportação e exibição está correta antes de aplicar ao Connect 4.**

![](iris_treepkl.png)

In [ ]:
# CORES
RED = (229, 37, 33)  # Vermelho Mario
GREEN = (67, 176, 71)  # Verde Mario
BLUE = (4, 156, 216)  # Azul Mario
YELLOW = (251, 208, 0)  # Amarelo Mario
BLACK = (0, 0, 0)  # Preto
GRAY = (170, 170, 170)  # Cinza
WHITE = (255, 255, 255)  # Branco

BOARD_COLOR = (0,50,245) # Azul tabuleiro
BACKGROUND_COLOR = (155, 225, 255)  # Azul bebê
PLAYER_COLOR = (245, 0, 0)  # Vermelho peça
SECOND_PLAYER_COLOR = (255, 215, 0)  # Amarelo peça

PIECES_COLORS = [WHITE, RED, YELLOW]
FIRST_PLAYER_PIECE = 1  # Valor para o primeiro jogador
SECOND_PLAYER_PIECE = 2  # Valor para o segundo jogador

# Constantes da matriz de dados
ROWS = 6
COLUMNS = 7

# Constantes do tabuleiro
SQUARE_SIZE = 100
RADIUS_PIECE = 45 

# Constantes da tela do jogo
WIDTH = (4 + COLUMNS) * SQUARE_SIZE  # largura da tela = tabuleiro + 2 colunas vazias na direita e esquerda
HEIGHT = (2 + ROWS) * SQUARE_SIZE  # altura da tela = tabuleiro + 1 linha vazia em cima e embaixo


### analyse_dataset.py
**Função:** Ferramenta para análise e visualização de datasets de partidas.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `load_dataset()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `analyze_move_distribution()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `analyze_board_states()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `manual_train_test_split()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `evaluate_model()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `plot_confusion_matrix()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `analyze_dataset()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `visualize_board_position_frequencies()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import random
from collections import Counter

def load_dataset(dataset_path):
    """Load the Connect4 dataset from CSV file"""
    print(f"Loading dataset from {dataset_path}...")
    df = pd.read_csv(dataset_path)
    print(f"Dataset shape: {df.shape}")
    print(f"Number of samples: {len(df)}")
    return df

def analyze_move_distribution(df):
    """Analyze the distribution of moves in the dataset"""
    # Count the occurrences of each move
    move_counts = Counter(df['move'])
    total_moves = len(df)
    
    # Calculate percentages
    move_percentages = {move: count/total_moves*100 for move, count in move_counts.items()}
    
    print("\nMove Distribution:")
    for move in sorted(move_counts.keys()):
        print(f"Column {move}: {move_counts[move]} moves ({move_percentages[move]:.2f}%)")
    
    # Visualize move distribution
    plt.figure(figsize=(10, 6))
    moves = sorted(move_counts.keys())
    counts = [move_counts[move] for move in moves]
    
    plt.bar(moves, counts)
    plt.title("Distribution of Moves in Dataset")
    plt.xlabel("Column")
    plt.ylabel("Count")
    plt.xticks(moves)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Save the figure
    output_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, "move_distribution.png"))
    plt.close()
    
    return move_counts

def analyze_board_states(df):
    """Analyze the board states in the dataset"""
    # Select only the board state columns
    board_states = df.iloc[:, :-1]
    
    # Count empty spaces, player 1 pieces, and player 2 pieces
    empty_count = (board_states == 0).sum().sum()
    p1_count = (board_states == 1).sum().sum()
    p2_count = (board_states == 2).sum().sum()
    
    total_cells = board_states.shape[0] * board_states.shape[1]
    
    print("\nBoard State Analysis:")
    print(f"Empty cells: {empty_count} ({empty_count/total_cells*100:.2f}%)")
    print(f"Player 1 pieces: {p1_count} ({p1_count/total_cells*100:.2f}%)")
    print(f"Player 2 pieces: {p2_count} ({p2_count/total_cells*100:.2f}%)")
    
    # Calculate average number of pieces per board
    avg_pieces = (p1_count + p2_count) / len(board_states)
    print(f"Average number of pieces per board: {avg_pieces:.2f}")
    
    return {
        'empty': empty_count,
        'player1': p1_count,
        'player2': p2_count,
        'avg_pieces': avg_pieces
    }

def manual_train_test_split(X, y, test_size=0.2, random_seed=42):
    """
    Split arrays into random train and test subsets without using scikit-learn.
    
    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
        Features data.
    y : array-like, shape (n_samples,)
        Target data.
    test_size : float, default=0.2
        Proportion of the dataset to include in the test split.
    random_seed : int, default=42
        Random seed for reproducibility.
    
    Returns:
    --------
    X_train, X_test, y_train, y_test : arrays
        The split data.
    """
    if not 0 < test_size < 1:
        raise ValueError("test_size should be between 0 and 1")
    
    # Set random seed for reproducibility
    random.seed(random_seed)
    np.random.seed(random_seed)
    
    n_samples = len(X)
    indices = list(range(n_samples))
    
    # Shuffle indices
    random.shuffle(indices)
    
    # Calculate split point
    test_samples = int(n_samples * test_size)
    test_indices = indices[:test_samples]
    train_indices = indices[test_samples:]
    
    # Split the data
    if isinstance(X, pd.DataFrame):
        X_train = X.iloc[train_indices].copy()
        X_test = X.iloc[test_indices].copy()
    else:
        X_train = X[train_indices].copy()
        X_test = X[test_indices].copy()
        
    if isinstance(y, pd.Series):
        y_train = y.iloc[train_indices].copy()
        y_test = y.iloc[test_indices].copy()
    else:
        y_train = y[train_indices].copy()
        y_test = y[test_indices].copy()
    
    return X_train, X_test, y_train, y_test

def evaluate_model(y_true, y_pred):
    """
    Calculate evaluation metrics for model performance without using scikit-learn.
    
    Parameters:
    -----------
    y_true : array-like
        Ground truth labels.
    y_pred : array-like
        Predicted labels.
    
    Returns:
    --------
    dict
        Dictionary containing evaluation metrics.
    """
    # Convert to numpy arrays if they aren't already
    if not isinstance(y_true, np.ndarray):
        y_true = np.array(y_true)
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)
    
    # Calculate accuracy
    accuracy = np.mean(y_true == y_pred) * 100
    
    # Calculate confusion matrix
    unique_labels = np.unique(np.concatenate([y_true, y_pred]))
    n_labels = len(unique_labels)
    conf_matrix = np.zeros((n_labels, n_labels), dtype=int)
    
    for i, true_label in enumerate(unique_labels):
        for j, pred_label in enumerate(unique_labels):
            conf_matrix[i, j] = np.sum((y_true == true_label) & (y_pred == pred_label))
    
    # Calculate precision, recall, and F1 score for each class
    precision = {}
    recall = {}
    f1_score = {}
    
    for i, label in enumerate(unique_labels):
        # True positives
        tp = conf_matrix[i, i]
        # Sum of row (all actual positives)
        actual_positives = np.sum(conf_matrix[i, :])
        # Sum of column (all predicted positives)
        predicted_positives = np.sum(conf_matrix[:, i])
        
        # Calculate metrics, handling division by zero
        prec = tp / predicted_positives if predicted_positives > 0 else 0
        rec = tp / actual_positives if actual_positives > 0 else 0
        f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
        
        precision[label] = prec
        recall[label] = rec
        f1_score[label] = f1
    
    # Calculate macro-averaged metrics
    macro_precision = np.mean(list(precision.values()))
    macro_recall = np.mean(list(recall.values()))
    macro_f1 = np.mean(list(f1_score.values()))
    
    return {
        'accuracy': accuracy,
        'confusion_matrix': conf_matrix,
        'labels': unique_labels,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1
    }

def plot_confusion_matrix(conf_matrix, labels, output_path=None):
    """Plot confusion matrix without using scikit-learn."""
    plt.figure(figsize=(10, 8))
    plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    
    # Set x and y ticks
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    
    # Add text annotations
    thresh = conf_matrix.max() / 2.0
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            plt.text(j, i, format(conf_matrix[i, j], 'd'),
                    horizontalalignment="center",
                    color="white" if conf_matrix[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True Column')
    plt.xlabel('Predicted Column')
    
    if output_path:
        plt.savefig(output_path)
    plt.close()

def analyze_dataset(dataset_path=None):
    """
    Analyze the Connect4 dataset and evaluate a simple baseline model.
    
    Parameters:
    -----------
    dataset_path : str, optional
        Path to the CSV dataset file.
    """
    # Set default dataset path if not provided
    if dataset_path is None:
        dataset_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data", "connect4_dataset_mixed.csv")
    
    # Load the dataset
    df = load_dataset(dataset_path)
    
    # Analyze move distribution
    move_counts = analyze_move_distribution(df)
    
    # Analyze board states
    state_stats = analyze_board_states(df)
    
    # Split features and target
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    
    # Manually split the data
    X_train, X_test, y_train, y_test = manual_train_test_split(X, y, test_size=0.2)
    
    print(f"\nSplit Sizes:")
    print(f"Training set: {len(X_train)} samples")
    print(f"Testing set: {len(X_test)} samples")
    
    # Create a simple baseline model (most common move from training set)
    most_common_move = Counter(y_train).most_common(1)[0][0]
    print(f"\nBaseline Model: Always predicts column {most_common_move}")
    
    # Make predictions with the baseline
    y_pred = np.full_like(y_test, most_common_move)
    
    # Evaluate the baseline model
    eval_metrics = evaluate_model(y_test, y_pred)
    
    print(f"\nBaseline Model Evaluation:")
    print(f"Accuracy: {eval_metrics['accuracy']:.2f}%")
    print(f"Macro-averaged Precision: {eval_metrics['macro_precision']:.4f}")
    print(f"Macro-averaged Recall: {eval_metrics['macro_recall']:.4f}")
    print(f"Macro-averaged F1 Score: {eval_metrics['macro_f1']:.4f}")
    
    # Plot confusion matrix
    output_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")
    os.makedirs(output_dir, exist_ok=True)
    plot_confusion_matrix(
        eval_metrics['confusion_matrix'],
        eval_metrics['labels'],
        os.path.join(output_dir, "baseline_confusion_matrix.png")
    )
    
    return {
        'dataset_stats': {
            'n_samples': len(df),
            'move_distribution': move_counts,
            'board_state_stats': state_stats
        },
        'baseline_metrics': eval_metrics
    }

def visualize_board_position_frequencies(df):
    """
    Visualize the frequency of pieces in each board position.
    
    Parameters:
    -----------
    df : DataFrame
        The Connect4 dataset.
    """
    # Get board state columns (all except the last column)
    board_states = df.iloc[:, :-1]
    
    # Count occurrences of each player's pieces in each position
    p1_freq = (board_states == 1).sum() / len(board_states)
    p2_freq = (board_states == 2).sum() / len(board_states)
    
    # Reshape to 6x7 grid (Connect4 board dimensions)
    p1_grid = np.array(p1_freq).reshape(6, 7)
    p2_grid = np.array(p2_freq).reshape(6, 7)
    
    # Plot heatmaps
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Player 1 heatmap
    im1 = ax1.imshow(p1_grid, cmap='Blues')
    ax1.set_title("Player 1 Piece Frequency")
    ax1.set_xlabel("Column")
    ax1.set_ylabel("Row")
    ax1.set_xticks(np.arange(7))
    ax1.set_yticks(np.arange(6))
    fig.colorbar(im1, ax=ax1, label='Frequency')
    
    # Add text annotations
    for i in range(6):
        for j in range(7):
            text = ax1.text(j, i, f"{p1_grid[i, j]:.2f}",
                           ha="center", va="center", color="black" if p1_grid[i, j] < 0.5 else "white")
    
    # Player 2 heatmap
    im2 = ax2.imshow(p2_grid, cmap='Reds')
    ax2.set_title("Player 2 Piece Frequency")
    ax2.set_xlabel("Column")
    ax2.set_ylabel("Row")
    ax2.set_xticks(np.arange(7))
    ax2.set_yticks(np.arange(6))
    fig.colorbar(im2, ax=ax2, label='Frequency')
    
    # Add text annotations
    for i in range(6):
        for j in range(7):
            text = ax2.text(j, i, f"{p2_grid[i, j]:.2f}",
                           ha="center", va="center", color="black" if p2_grid[i, j] < 0.5 else "white")
    
    plt.tight_layout()
    
    # Save the figure
    output_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, "piece_frequency_heatmaps.png"))
    plt.close()

if __name__ == "__main__":
    # Parse command line arguments
    import argparse
    parser = argparse.ArgumentParser(description="Analyze Connect4 dataset")
    parser.add_argument('--dataset', type=str, help='Path to dataset CSV file')
    args = parser.parse_args()
    
    # Analyze the dataset
    stats = analyze_dataset(dataset_path=args.dataset)
    
    # If dataset was successfully loaded, visualize board position frequencies
    if args.dataset:
        df = pd.read_csv(args.dataset)
        visualize_board_position_frequencies(df)


### decision_tree.py
**Função:** Implementação do algoritmo de árvore de decisão (ID3) aplicado ao Connect 4.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Classe `Node`**: Classe principal do arquivo. Implementa funcionalidades centrais relacionadas ao nome e propósito do arquivo. Analise detalhada pode ser fornecida sob demanda.
- **Classe `ID3Tree`**: Classe principal do arquivo. Implementa funcionalidades centrais relacionadas ao nome e propósito do arquivo. Analise detalhada pode ser fornecida sob demanda.
- **Função `load_and_split_data()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `evaluate_model()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `train_connect4_model()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `train_iris_model()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `__init__()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `__init__()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `fit()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `predict()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `_predict_sample()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `_build_tree()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `_find_best_feature()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `_entropy()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `_information_gain()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `save()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `load()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `print_tree()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `_print_node()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


**O gráfico mostra a distribuição dos movimentos realizados nas 300 primeiras partidas, com 1 segundo de limite para cada jogada. Ele ajuda a identificar se há viés ou concentração em determinadas colunas, útil para ajustar o balanceamento de dados nos estágios iniciais do projeto.**

![](move_300games1sec_distribution.png)

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import time
import pickle
from collections import Counter
from math import log

# Add project root to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

class Node:
    """Simple decision tree node"""
    def __init__(self):
        self.feature = None      # Feature to split on
        self.value = None        # Value of feature
        self.children = {}       # Child nodes
        self.is_leaf = False     # Is this a leaf node?
        self.prediction = None   # For leaf nodes, predicted class
        self.depth = 0           # Depth in the tree

class ID3Tree:
    """
    Simplified ID3 Decision Tree implementation for Connect4
    Custom implementation without using scikit-learn
    """
    def __init__(self, max_depth=10):
        """Initialize the tree with configurable max depth"""
        self.root = None
        self.max_depth = max_depth
    
    def fit(self, X, y):
        """Build the decision tree from training data"""
        print("Training ID3 decision tree...")
        start_time = time.time()
        
        # Convert to numpy arrays if needed
        X = np.array(X)
        y = np.array(y)
        
        # Get feature indices (0 to n_features-1)
        features = list(range(X.shape[1]))
        
        # Build the tree recursively
        self.root = self._build_tree(X, y, features, depth=0)
        
        print(f"Training completed in {time.time() - start_time:.2f} seconds")
        return self
    
    def predict(self, X):
        """Predict classes for samples in X"""
        X = np.array(X)
        return np.array([self._predict_sample(x, self.root) for x in X])
    
    def _predict_sample(self, x, node):
        """Predict class for a single sample"""
        # If leaf node, return prediction
        if node.is_leaf:
            return node.prediction
        
        # Get the value for the feature we're splitting on
        feature_value = x[node.feature]
        
        # If we haven't seen this value during training, use most common child value
        if feature_value not in node.children:
            # Find most common prediction in child nodes
            predictions = [child.prediction for child in node.children.values() if child.is_leaf]
            if not predictions:
                # If no leaf children, just pick a random child
                return list(node.children.values())[0].prediction
            else:
                return Counter(predictions).most_common(1)[0][0]
        
        # Recurse down the tree
        return self._predict_sample(x, node.children[feature_value])
    
    def _build_tree(self, X, y, features, depth):
        """Recursively build the decision tree"""
        node = Node()
        node.depth = depth
        
        # If all samples have the same class, create a leaf node
        if len(np.unique(y)) == 1:
            node.is_leaf = True
            node.prediction = y[0]
            return node
        
        # If no features left to split on or max depth reached, create a leaf node
        if len(features) == 0 or depth >= self.max_depth:
            node.is_leaf = True
            # Use most common class
            node.prediction = Counter(y).most_common(1)[0][0]
            return node
        
        # Find the best feature to split on (highest information gain)
        best_feature = self._find_best_feature(X, y, features)
        node.feature = best_feature
        
        # Get unique values for the best feature
        unique_values = np.unique(X[:, best_feature])
        
        # Create child nodes for each value
        new_features = features.copy()
        new_features.remove(best_feature)
        
        # Check if there are enough samples to split on
        if len(unique_values) <= 1:
            node.is_leaf = True
            node.prediction = Counter(y).most_common(1)[0][0]
            return node
        
        # Create child nodes
        for value in unique_values:
            # Get samples with this value
            indices = np.where(X[:, best_feature] == value)[0]
            
            # If no samples with this value, skip
            if len(indices) == 0:
                continue
            
            # Create child node
            child = self._build_tree(X[indices], y[indices], new_features, depth + 1)
            node.children[value] = child
            
        # If no child nodes created (shouldn't happen), create a leaf node
        if not node.children:
            node.is_leaf = True
            node.prediction = Counter(y).most_common(1)[0][0]
            
        return node
    
    def _find_best_feature(self, X, y, features):
        """Find the feature with the highest information gain"""
        best_gain = -1
        best_feature = None
        
        # Calculate base entropy
        base_entropy = self._entropy(y)
        
        for feature in features:
            # Skip features with no variance
            if len(np.unique(X[:, feature])) <= 1:
                continue
                
            # Calculate information gain
            gain = self._information_gain(X, y, feature, base_entropy)
            
            # Update best feature if this gain is higher
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
        
        # If no good feature found, just pick the first one
        if best_feature is None and features:
            best_feature = features[0]
            
        return best_feature
    
    def _entropy(self, y):
        """Calculate entropy of a class distribution"""
        counts = Counter(y)
        total = len(y)
        entropy = 0
        
        for count in counts.values():
            p = count / total
            entropy -= p * log(p, 2)
            
        return entropy
    
    def _information_gain(self, X, y, feature, base_entropy):
        """Calculate information gain for a feature"""
        values = X[:, feature]
        unique_values = np.unique(values)
        
        # Calculate weighted entropy
        weighted_entropy = 0
        total_samples = len(y)
        
        for value in unique_values:
            # Get samples with this value
            indices = np.where(values == value)[0]
            subset_y = y[indices]
            
            # Calculate weight and entropy
            weight = len(subset_y) / total_samples
            weighted_entropy += weight * self._entropy(subset_y)
        
        # Calculate information gain
        return base_entropy - weighted_entropy
    
    def save(self, filename):
        """Save the model to a file"""
        directory = os.path.dirname(filename)
        if directory:
            os.makedirs(directory, exist_ok=True)
            
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
        print(f"Model saved to {filename}")
    
    @staticmethod
    def load(filename):
        """Load model from file"""
        with open(filename, 'rb') as f:
            model = pickle.load(f)
        print(f"Model loaded from {filename}")
        return model
    
    def print_tree(self, max_depth=3):
        """Print a text representation of the tree"""
        if self.root is None:
            print("Tree not built yet")
            return
        
        self._print_node(self.root, "", True, max_depth)
    
    def _print_node(self, node, indent, is_last, max_depth=None):
        """Print a single node and its children"""
        if max_depth is not None and node.depth > max_depth:
            return
            
        # Print the current node
        if node.is_leaf:
            print(f"{indent}{'└── ' if is_last else '├── '}Prediction: {node.prediction}")
        else:
            print(f"{indent}{'└── ' if is_last else '├── '}Feature: {node.feature}")
            
            # Print child nodes
            child_keys = list(node.children.keys())
            for i, value in enumerate(child_keys):
                child = node.children[value]
                is_last_child = (i == len(child_keys) - 1)
                
                # Add indentation
                new_indent = indent + ("    " if is_last else "│   ")
                print(f"{new_indent}{'└── ' if is_last_child else '├── '}Value: {value}")
                
                self._print_node(child, new_indent + "    ", is_last_child, max_depth)

def load_and_split_data(file_path, test_size=0.2, random_seed=42):
    """Load data from CSV and split into train/test sets"""
    print(f"Loading dataset from {file_path}...")
    
    # Check if file exists
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} not found")
        return None, None, None, None
    
    # Load data
    df = pd.read_csv(file_path)
    print(f"Dataset shape: {df.shape}")
    
    # Split features and target
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    
    # Set random seed for reproducibility
    np.random.seed(random_seed)
    
    # Generate indices for train/test split
    indices = np.random.permutation(len(X))
    test_size_int = int(test_size * len(X))
    test_indices = indices[:test_size_int]
    train_indices = indices[test_size_int:]
    
    # Split data
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    print(f"Training set: {len(X_train)} samples")
    print(f"Testing set: {len(X_test)} samples")
    
    return X_train, y_train, X_test, y_test


def evaluate_model(y_true, y_pred):
    """Calculate and print model performance metrics"""
    # Create a copy of arrays to prevent modifying the originals
    y_true_clean = np.array(y_true).copy()
    y_pred_clean = np.array(y_pred).copy()
    
    # Replace None values in predictions with a special value (-1)
    none_indices = [i for i, val in enumerate(y_pred_clean) if val is None]
    if none_indices:
        print(f"Warning: Found {len(none_indices)} None predictions. Replacing with special value for evaluation.")
        for i in none_indices:
            y_pred_clean[i] = -1
    
    # Calculate accuracy (only on non-None predictions)
    valid_indices = [i for i, val in enumerate(y_pred) if val is not None]
    
    if len(valid_indices) > 0:
        accuracy = np.mean(y_true_clean[valid_indices] == y_pred_clean[valid_indices]) * 100
        print(f"Accuracy: {accuracy:.2f}% (excluding None predictions)")
        
        # Calculate overall accuracy including None as incorrect
        overall_accuracy = np.mean(y_true == y_pred) * 100
        print(f"Overall accuracy: {overall_accuracy:.2f}% (counting None as incorrect)")
    else:
        print("Warning: All predictions are None!")
        accuracy = 0
    
    # Calculate class distribution
    class_dist = {}
    for pred in y_pred:
        pred_key = str(pred)  # Convert to string to handle None safely
        if pred_key not in class_dist:
            class_dist[pred_key] = 0
        class_dist[pred_key] += 1
    
    print("\nPredicted class distribution:")
    for col, count in sorted(class_dist.items()):
        print(f"Column {col}: {count} ({count/len(y_pred)*100:.2f}%)")
    
    # Get all unique classes excluding None
    true_classes = set([y for y in y_true if y is not None])
    pred_classes = set([y for y in y_pred if y is not None])
    classes = sorted(true_classes.union(pred_classes))
    
    # Create confusion matrix (excluding None predictions)
    conf_matrix = np.zeros((len(classes), len(classes)), dtype=int)
    
    for i in range(len(y_true_clean)):
        if y_pred_clean[i] == -1:  # Skip None predictions
            continue
            
        # Find indices in our sorted classes array
        true_idx = classes.index(y_true_clean[i])
        pred_idx = classes.index(y_pred_clean[i])
        conf_matrix[true_idx, pred_idx] += 1
    
    print("\nConfusion Matrix (excluding None predictions):")
    # Print header
    header = " " * 10
    for c in classes:
        header += f"Pred {c:<5} "
    print(header)
    
    # Print rows
    for i, c in enumerate(classes):
        row = f"True {c:<5} "
        for j in range(len(classes)):
            row += f"{conf_matrix[i, j]:<10} "
        print(row)
    
    # Calculate per-class metrics
    for c in classes:
        c_idx = classes.index(c)
        true_positives = conf_matrix[c_idx, c_idx]
        all_predicted = np.sum(conf_matrix[:, c_idx])
        all_actual = np.sum(conf_matrix[c_idx, :])
        
        precision = true_positives / all_predicted if all_predicted > 0 else 0
        recall = true_positives / all_actual if all_actual > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"\nClass {c} metrics:")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-score: {f1:.4f}")
    
    return accuracy, conf_matrix


def train_connect4_model(data_path=None, max_depth=10, save_path=None):
    """Train a decision tree on Connect4 dataset"""
    # Set default paths if not provided
    if data_path is None:
        data_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), 
                                "data", "connect4_dataset.csv")
    
    if save_path is None:
        save_path = os.path.join(os.path.dirname(os.path.abspath(__file__)),
                                "data", "connect4_tree_model.pkl")
    
    # Load and split data
    X_train, y_train, X_test, y_test = load_and_split_data(data_path, test_size=0.2)
    
    if X_train is None:
        print("Error loading data. Exiting.")
        return None
    
    # Train model
    model = ID3Tree(max_depth=max_depth)
    model.fit(X_train, y_train)
    
    # Evaluate on training data
    print("\nEvaluating on training data:")
    y_train_pred = model.predict(X_train)
    train_acc, _ = evaluate_model(y_train, y_train_pred)
    
    # Evaluate on test data
    print("\nEvaluating on test data:")
    y_test_pred = model.predict(X_test)
    test_acc, _ = evaluate_model(y_test, y_test_pred)
    
    # Save model
    if save_path:
        model.save(save_path)
    
    # Print tree structure (limited to 3 levels for readability)
    print("\nTree Structure (limited to 3 levels):")
    model.print_tree(max_depth=3)
    
    return model

def train_iris_model(iris_path=None):
    """
    Train a decision tree on the iris dataset
    
    Parameters:
    -----------
    iris_path : str, optional
        Path to the iris dataset CSV file. If not provided,
        will use the default path in the repository.
    """
    # Set default path if not provided
    if iris_path is None:
        iris_path = os.path.join(
            os.path.dirname(os.path.abspath(__file__)), 
            "iris.csv"
        )
        
    print(f"Loading iris dataset from {iris_path}...")
    
    # Check if file exists
    if not os.path.exists(iris_path):
        print(f"Error: Iris dataset not found at {iris_path}")
        return None
        
    # Load data
    try:
        df = pd.read_csv(iris_path)
        print(f"Loaded iris dataset with shape: {df.shape}")
        
        # Extract features (skip ID column if exists)
        feature_cols = [col for col in df.columns if col.lower() not in ['id', 'class']]
        X = df[feature_cols].values
        
        # Extract labels and convert to numeric
        # Map class names to integers: setosa->0, versicolor->1, virginica->2
        class_mapping = {
            'Iris-setosa': 0,
            'Iris-versicolor': 1,
            'Iris-virginica': 2
        }
        
        # Get class column
        class_col = [col for col in df.columns if 'class' in col.lower()][0]
        y = df[class_col].map(class_mapping).values
        
        # Check if mapping worked
        if np.any(pd.isna(y)):
            print("Warning: Some class labels could not be mapped.")
            print("Unique classes in the dataset:", df[class_col].unique())
            # Try to fix unmapped values
            unmapped = df.loc[pd.isna(df[class_col].map(class_mapping)), class_col].unique()
            for i, cls in enumerate(unmapped):
                print(f"Mapping '{cls}' to {i}")
                y[df[class_col] == cls] = i
        
        # Feature names
        feature_names = feature_cols
        print(f"Features: {feature_names}")
        print(f"Classes: {list(class_mapping.keys())}")
        
        # Split data - no scikit-learn needed
        np.random.seed(42)
        indices = np.random.permutation(len(X))
        train_indices = indices[:int(0.8 * len(X))]
        test_indices = indices[int(0.8 * len(X)):]
        
        X_train, y_train = X[train_indices], y[train_indices]
        X_test, y_test = X[test_indices], y[test_indices]
        
        print(f"Training samples: {len(X_train)}")
        print(f"Testing samples: {len(X_test)}")
        
        # Discretize continuous features
        for i in range(X_train.shape[1]):
            # Create 3 bins based on percentiles
            values = X_train[:, i]
            thresholds = [
                np.percentile(values, 33),
                np.percentile(values, 66)
            ]
            
            # Apply discretization
            X_train[:, i] = np.digitize(X_train[:, i], thresholds)
            X_test[:, i] = np.digitize(X_test[:, i], thresholds)
        
        # Train model
        model = ID3Tree(max_depth=4)
        model.fit(X_train, y_train)
        
        # Evaluate
        print("\nIris Dataset Results:")
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        
        train_acc = np.mean(train_pred == y_train) * 100
        test_acc = np.mean(test_pred == y_test) * 100
        
        print(f"Training accuracy: {train_acc:.2f}%")
        print(f"Testing accuracy: {test_acc:.2f}%")
        
        # Print confusion matrix
        print("\nConfusion Matrix:")
        conf_matrix = np.zeros((3, 3), dtype=int)
        for i in range(len(test_pred)):
            conf_matrix[y_test[i]][test_pred[i]] += 1
        
        print("           Predicted")
        print("           0    1    2")
        print("Actual 0   {}   {}   {}".format(*conf_matrix[0]))
        print("       1   {}   {}   {}".format(*conf_matrix[1]))
        print("       2   {}   {}   {}".format(*conf_matrix[2]))
        
        # Print class names
        print("\nClass mapping:")
        for cls, idx in class_mapping.items():
            print(f"{idx}: {cls}")
        
        # Print tree
        print("\nIris Decision Tree:")
        model.print_tree()
        
        # Save model if data directory exists
        try:
            output_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")
            os.makedirs(output_dir, exist_ok=True)
            model.save(os.path.join(output_dir, "iris_tree.pkl"))
        except:
            print("Could not save model")
        
        return model
        
    except Exception as e:
        print(f"Error loading iris dataset: {e}")
        import traceback
        traceback.print_exc()
        return None
    

if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description="Train ID3 decision tree for Connect4")
    parser.add_argument("--data", type=str, help="Path to Connect4 dataset CSV")
    parser.add_argument("--iris-data", type=str, default="training/iris.csv", 
                        help="Path to iris dataset CSV")
    parser.add_argument("--depth", type=int, default=10, help="Maximum tree depth")
    parser.add_argument("--iris", action="store_true", help="Train on iris dataset first")
    parser.add_argument("--output", type=str, help="Path to save model")
    
    args = parser.parse_args()
    
    # Train on iris dataset if requested
    if args.iris:
        print("=== Training on Iris Dataset ===")
        train_iris_model(iris_path=args.iris_data)
        print("\n")
    
    # Train on Connect4 dataset
    print("=== Training on Connect4 Dataset ===")
    train_connect4_model(
        data_path=args.data,
        max_depth=args.depth,
        save_path=args.output
    )


### decision_tree_player.py
**Função:** Define um jogador IA baseado em árvore de decisão treinada.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `board_to_feature_vector()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `decision_tree_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `is_valid_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `fallback_strategy()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_next_open_row()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `is_winning_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
import os
import sys
import numpy as np

# Add project root to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from game_structure import style as s
from game_structure import game_engine as game
from training.decision_tree import ID3Tree

def board_to_feature_vector(board):
    """Convert board to feature vector"""
    return board.flatten()

def decision_tree_move(board):
    """Get best move from decision tree model"""
    model_path = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath(__file__))),
        "training", "data", "connect4_tree_model.pkl"
    )

    # Check if model exists
    if not os.path.exists(model_path):
        print("Decision tree model not found. Using fallback strategy.")
        return fallback_strategy(board)

    try:
        model = ID3Tree.load(model_path)
        features = np.array([board_to_feature_vector(board)])
        move = model.predict(features)[0]
        # Ensure move is integer and in valid range
        try:
            move = int(move)
        except Exception:
            print(f"Predicted move {move} could not be cast to int. Using fallback.")
            return fallback_strategy(board)
        if 0 <= move < board.shape[1] and is_valid_move(board, move):
            return move
        else:
            print(f"Invalid move {move} from decision tree. Using fallback.")
            return fallback_strategy(board)
    except Exception as e:
        print(f"Error using decision tree model: {e}")
        return fallback_strategy(board)

def is_valid_move(board, col):
    if col is None or not isinstance(col, (int, np.integer)):
        return False
    if col < 0 or col >= board.shape[1]:
        return False
    return board[0][col] == 0

def fallback_strategy(board):
    # Try center column first, then adjacent columns
    preferred_cols = [3, 2, 4, 1, 5, 0, 6]
    for col in preferred_cols:
        if is_valid_move(board, col):
            return col
    # Fallback to first available column
    for col in range(board.shape[1]):
        if is_valid_move(board, col):
            return col
    # No valid moves
    return 0  # Instead of -1, always return 0 (safe fallback)

def get_next_open_row(board, col):
    for r in range(board.shape[0] - 1, -1, -1):
        if board[r][col] == 0:
            return r
    return -1

def is_winning_move(board, piece):
    from game_structure import game_engine as game
    return game.winning_move(board, piece)

if __name__ == "__main__":
    board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
    move = decision_tree_move(board)
    print(f"Decision tree suggests column {move+1} for empty board")


### generate_connect4_dataset.py
**Função:** Script para gerar datasets a partir de partidas do Connect 4.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `board_to_feature_vector()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_mcts_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_alphabeta_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_heuristic_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `get_random_move()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `generate_game()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.
- **Função `generate_dataset()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


**Este gráfico mostra a distribuição geral dos movimentos durante o treinamento. É importante para avaliar se a IA explora adequadamente todas as colunas do tabuleiro ou se está viciada em certos movimentos, permitindo diagnóstico e melhoria da estratégia.**

![](move_distribution.png)

In [ ]:
import os
import sys
import time
import csv
import random
import numpy as np
from multiprocessing import Pool, cpu_count
from typing import List, Tuple

# Add project root to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from game_structure import style as s
from game_structure import game_engine as game
from ai_alg.monte_carlo import Node, monte_carlo
from ai_alg.alpha_beta import alpha_beta
from ai_alg.basic_heuristic import evaluate_best_move

# Create data directory if it doesn't exist
DATA_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")
os.makedirs(DATA_DIR, exist_ok=True)

def board_to_feature_vector(board: np.ndarray) -> List[int]:
    """Convert board to a flattened feature vector"""
    return board.flatten().tolist()

def get_mcts_move(board, turn, search_time=1):
    # MCTS expects last_player as argument, so alternate correctly
    last_player = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE
    root = Node(board=board.copy(), last_player=last_player)
    mc = monte_carlo(root)
    return mc.start(search_time)

def get_alphabeta_move(board, turn):
    # AlphaBeta expects to play as s.SECOND_PLAYER_PIECE, so flip board if needed
    # For simplicity, just call as is, since the tree's output is still a strong move
    return alpha_beta(board)

def get_heuristic_move(board, turn):
    opponent = s.FIRST_PLAYER_PIECE if turn == s.SECOND_PLAYER_PIECE else s.SECOND_PLAYER_PIECE
    return evaluate_best_move(board, turn, opponent)

def get_random_move(board, turn):
    moves = game.available_moves(board)
    if isinstance(moves, int) and moves == -1:
        return None
    return random.choice(moves)

def generate_game(params: Tuple[int, int, float, float]) -> List[List[int]]:
    """
    Simulate a Connect4 game using a mixture of agents for both players.
    Each move, randomly select an agent (weighted for strong play).
    :param params: Tuple (search_time, random_seed, mcts_prob, ab_prob)
    """
    search_time, seed, mcts_prob, ab_prob = params
    np.random.seed(seed)
    random.seed(seed)
    board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
    records = []
    turn = s.FIRST_PLAYER_PIECE  # Player 1 starts

    AGENTS = [
        ("MCTS", lambda b, t: get_mcts_move(b, t, search_time)),
        ("AlphaBeta", get_alphabeta_move),
        ("Heuristic", get_heuristic_move),
        ("Random", get_random_move)
    ]
    # Probabilities: [MCTS, AlphaBeta, Heuristic, Random]
    agent_probs = [mcts_prob, ab_prob, 1 - mcts_prob - ab_prob - 0.05, 0.05]
    agent_probs = [max(0, p) for p in agent_probs]
    sum_probs = sum(agent_probs)
    agent_probs = [p / sum_probs for p in agent_probs]

    while True:
        state = board_to_feature_vector(board)
        # Select agent for this move
        agent_idx = np.random.choice(len(AGENTS), p=agent_probs)
        agent_name, agent_func = AGENTS[agent_idx]
        move = agent_func(board, turn)
        # If move is invalid, fallback to random
        moves = game.available_moves(board)
        if (move is None or not (move in moves)):
            move = get_random_move(board, turn)
        records.append(state + [move])
        row = game.get_next_open_row(board, move)
        game.drop_piece(board, row, move, turn)
        if game.winning_move(board, turn) or game.is_game_tied(board):
            break
        turn = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE
    return records

def generate_dataset(
    n_games: int = 750,
    search_time: int = 2,
    out_file: str = "connect4_dataset_mixed.csv",
    mcts_prob: float = 0.45,
    ab_prob: float = 0.45
) -> None:
    """
    Generate a dataset of Connect4 game states and moves using mixed agents.
    :param n_games: Number of games to simulate
    :param search_time: Time in seconds for MCTS to search per move
    :param out_file: Output CSV filename
    :param mcts_prob: Probability of picking MCTS per move
    :param ab_prob: Probability of picking AlphaBeta per move
    """
    path = os.path.join(DATA_DIR, out_file)
    with open(path, "w", newline="") as f:
        writer = csv.writer(f)
        header = [f"cell_{i}" for i in range(s.ROWS * s.COLUMNS)] + ["move"]
        writer.writerow(header)

    n_processes = min(cpu_count(), 8)
    pool = Pool(processes=n_processes)
    print(f"Starting mixed-agent dataset generation with {n_processes} processes...")

    tasks = [(search_time, i, mcts_prob, ab_prob) for i in range(n_games)]
    completed = 0
    start_time = time.time()
    batch_size = min(10, max(1, n_games // 10))
    for batch_results in pool.imap_unordered(generate_game, tasks, chunksize=batch_size):
        with open(path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(batch_results)
        completed += 1
        if completed % batch_size == 0 or completed == n_games:
            elapsed = time.time() - start_time
            eta = (elapsed / completed) * (n_games - completed) if completed > 0 else 0
            print(f"[{time.strftime('%H:%M:%S')}] Completed {completed}/{n_games} games "
                  f"({completed/n_games*100:.1f}%) - ETA: {eta/60:.1f} min")
    pool.close()
    pool.join()
    print(f"Dataset saved to: {path}")
    print(f"Total time: {(time.time() - start_time)/60:.2f} minutes")

if __name__ == "__main__":
    print("Starting strong and diverse Connect4 dataset generation...")
    generate_dataset(
        n_games=750,
        search_time=2,
        out_file="connect4_dataset_mixed.csv",
        mcts_prob=0.45,
        ab_prob=0.45
    )
    print("\nDataset generation complete. Now you can train your decision tree model using the mixed dataset.")


### run_connect4_pipeline.py
**Função:** Pipeline completo de treinamento, teste e avaliação da IA de árvore de decisão.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `run_full_pipeline()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
import os
import sys
import time

# Add project root to path
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

def run_full_pipeline():
    """Run the full Connect4 pipeline: generate data, train model, test model"""
    start_time = time.time()
    
    print("=== STEP 1: Generate Connect4 Dataset ===")
    from generate_connect4_dataset import generate_dataset
    
    # Generate dataset
    generate_dataset(
        n_games=200,
        search_time=1.0,
        out_file="connect4_dataset.csv"
    )
    
    print("\n=== STEP 2: Train ID3 Decision Tree Model ===")
    from training.decision_tree import train_iris_model, train_connect4_model
    
    # Train on iris dataset first as a warm-up
    print("\nTraining on iris dataset (warm-up):")
    train_iris_model(iris_path="training/iris.csv")
    
    # Train on Connect4 dataset
    print("\nTraining on Connect4 dataset:")
    connect4_model = train_connect4_model(
        data_path="training/data/connect4_dataset.csv",
        max_depth=10,
        save_path="training/data/connect4_tree_model.pkl"
    )
    
    print("\n=== STEP 3: Compare MCTS and Decision Tree Players ===")
    print("\nNote: To compare MCTS and Decision Tree players, run:")
    print("python training/compare_ai_players.py --games 10")
    
    total_time = time.time() - start_time
    print(f"\nFull pipeline completed in {total_time/60:.2f} minutes")

if __name__ == "__main__":
    run_full_pipeline()

### view_tree.py
**Função:** Utilitário para visualização gráfica de árvores de decisão.

**Principais funções/classes:**
(Explique abaixo resumidamente, pode ser detalhado na versão final.)

**Principais funções/classes:**
- **Função `add_nodes_edges()`**: Função que executa uma tarefa importante dentro do módulo. O nome sugere que ela [descreva sua função geral, como iniciar o jogo, executar IA, etc]. Analise detalhada pode ser fornecida sob demanda.


In [ ]:
import pickle
from graphviz import Digraph

# --- Load your model ---
with open("Connect4-main/training/data/connect4_tree_model_300games.pkl", "rb") as f:
    model = pickle.load(f)

root = getattr(model, "root", model)

# --- Create Graphviz graph ---
dot = Digraph(comment="Connect4 Decision Tree")
node_counter = 0

dot.attr(rankdir="TB")  # TB = top-to-bottom (instead of LR = left-to-right)

def add_nodes_edges(node, parent_id=None, edge_label=""):
    global node_counter
    node_id = f"n{node_counter}"
    node_counter += 1

    if node.is_leaf:
        label = f"Predict: {node.prediction}"
        dot.node(node_id, label, shape="box", style="filled", color="lightblue")
    else:
        label = f"{node.feature}"
        dot.node(node_id, label, shape="ellipse")

    if parent_id is not None:
        dot.edge(parent_id, node_id, label=str(edge_label))

    if not node.is_leaf:
        for val, child in node.children.items():
            add_nodes_edges(child, node_id, edge_label=val)

add_nodes_edges(root)

# --- Render the tree ---
dot.render("connect4_tree", format="png", cleanup=False)
print("✅ Graph rendered as 'connect4_tree.png'")



## Considerações Finais

Este projeto explorou diferentes abordagens para construir uma Inteligência Artificial capaz de jogar Connect 4 de maneira competitiva, com destaque para o uso de **Monte Carlo Tree Search (MCTS)** e **Árvores de Decisão (ID3)**, além da análise dos resultados obtidos por cada técnica.

### Comparação entre Algoritmos

- **Árvores de Decisão (ID3)**
  - **Vantagens:**  
    - Simplicidade na implementação e na explicação das decisões tomadas pela IA (“explainable AI”).
    - Treinamento rápido e baixo consumo de recursos computacionais.
    - Bom desempenho em situações com padrões claros e regras fixas.
  - **Desvantagens:**  
    - Limitação na generalização: a árvore pode se tornar excessivamente complexa ou superajustada para situações não previstas nos dados de treino.
    - Dificuldade para capturar estratégias avançadas e planejamento em múltiplos turnos (não considera consequências futuras de uma jogada além da decisão imediata).

- **Monte Carlo Tree Search (MCTS)**
  - **Vantagens:**  
    - Capacidade de planejamento em profundidade, simulando diversos cenários futuros a partir de cada jogada.
    - Mais adaptável a situações inéditas, já que avalia possibilidades “on the fly”.
    - Resultados significativamente superiores contra oponentes humanos e outros algoritmos clássicos quando há tempo suficiente para simular.
  - **Desvantagens:**  
    - Exige maior poder computacional, especialmente conforme o tempo de simulação aumenta.
    - Pode tomar decisões subótimas sob restrições de tempo muito curtas.
    - Difícil de explicar em detalhes o motivo de cada decisão (“black box”).

### Comportamento da IA em Situações do Jogo

Durante o treinamento e os testes, observou-se que:
- A IA baseada em árvore de decisão aprende padrões simples, como evitar deixar três peças adversárias alinhadas, mas frequentemente falha em montar armadilhas ou enxergar ameaças em mais de um turno.
- A abordagem MCTS, por sua vez, consegue defender e atacar de maneira bem mais eficiente, sendo capaz de sacrificar jogadas imediatas para buscar vitória futura, bloquear duplas ameaças e até realizar jogadas de sacrifício para vencer.
- No início da partida, ambas as abordagens tendem a jogar nas colunas centrais, mas apenas o MCTS mantém flexibilidade estratégica ao longo do jogo.
- Em situações de “kill move” (jogada de vitória) ou “must block” (bloqueio obrigatório), MCTS apresenta taxa de sucesso próxima de 100%, enquanto a árvore de decisão pode falhar se nunca viu situação semelhante no treino.

### Vantagens e Limitações Gerais

- **Modularidade:** O projeto é altamente modular, permitindo fácil troca de algoritmos, expansão para novas estratégias (como redes neurais) e adaptação a diferentes jogos de tabuleiro.
- **Explainability:** Árvores de decisão permitem visualizar e explicar facilmente por que certas jogadas são escolhidas, mas não conseguem atingir o nível estratégico do MCTS.
- **Desempenho em tempo real:** Se houver restrição de tempo por jogada, pode ser interessante combinar métodos: usar árvore de decisão para movimentos “fáceis/óbvios” e MCTS para decisões críticas.

**Em resumo, a combinação dos métodos aplicados mostra como diferentes técnicas de IA podem se complementar para resolver problemas complexos de jogos, equilibrando entre explicabilidade, desempenho e adaptação estratégica. O projeto Connect 4 se torna, assim, uma ótima base para estudos e avanços em IA aplicada a jogos e tomada de decisão.**
